In [ ]:
import os 

import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import DATA_PATH, SHP_PATH
from energyemissionsregio.utils import solve_proxy_equation, get_proxy_var_list
from energyemissionsregio.disaggregation import perform_proxy_based_disaggregation
from energyemissionsregio.plotting_functions import plot_validation_data
from sklearn.metrics import mean_absolute_error

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cwd = os.getcwd()

In [ ]:
lau_shp = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))
lau_shp = lau_shp[lau_shp["code"].str.startswith(("DE", "ES"))]

### Commerce FEC (Eurostat)

In [ ]:
eurostat_data_nuts0 = pd.read_csv(os.path.join(DATA_PATH, f"final_energy_consumption_in_commerce.csv"))
eurostat_data_nuts0 = eurostat_data_nuts0[eurostat_data_nuts0["region_code"].str.startswith(("DE", "ES"))][
    ["region_code", "value"]].copy()

In [ ]:
eurostat_data_nuts0

### Hotmaps data

In [ ]:
validation_data = pd.read_csv(
        os.path.join(DATA_PATH, "heat_demand_non_residential.csv")
    )
validation_data = validation_data[validation_data["region_code"].str.startswith(("DE", "ES"))][
    ["region_code", "value"]].copy()

In [ ]:
target_data = validation_data.copy()
target_data["region_code"] = target_data["region_code"].str[:2]

target_data = target_data.groupby("region_code").sum().reset_index()

In [ ]:
target_data["value_confidence_level"] = 5

difference between the Eurostat data disaggregated and hotmaps data used for validation of disaggregation 

In [ ]:
diff_df = pd.merge(eurostat_data_nuts0, target_data, on="region_code")

In [ ]:
diff_df["diff"] = diff_df["value_x"] - diff_df["value_y"]

In [ ]:
diff_df

### Disaggregation of Hotmaps data

In [ ]:
proxy_equation = "de_non_residential_building_living_area*cproj_annual_mean_temperature_heating_degree_days"

In [ ]:
# read in proxy data 
proxy_var_list = get_proxy_var_list(proxy_equation)

proxy_data_dict = {}
for proxy_var in proxy_var_list:
    if os.path.exists(os.path.join(cwd, "..", "..", "data", "disaggregated_data", f"{proxy_var}.csv")):
        proxy_data = pd.read_csv(os.path.join(cwd, "..", "..", "data", "disaggregated_data", f"{proxy_var}.csv"))
    else:
        proxy_data = pd.read_csv(os.path.join(DATA_PATH, f"{proxy_var}.csv"))
        proxy_data["value_confidence_level"] = 5

    proxy_data = proxy_data[proxy_data["region_code"].str.startswith(("DE", "ES"))][["region_code", 
                                                                                     "value",
                                                                                     "value_confidence_level"]].copy()

    proxy_data["value"] = proxy_data["value"].fillna(0)
    proxy_data_dict.update({proxy_var: proxy_data})

solved_proxy_data = solve_proxy_equation(proxy_equation, proxy_data_dict)

disagg_data = perform_proxy_based_disaggregation(target_data, solved_proxy_data, "NUTS0", 4)

In [ ]:
# calulate MAE and country total -------------
merged_df_mae = pd.merge(validation_data, disagg_data, on = "region_code", how="outer", suffixes=("_true", "_disagg"))

true_values_de = merged_df_mae[merged_df_mae["region_code"].str.startswith("DE")]["value_true"]
disagg_values_de = merged_df_mae[merged_df_mae["region_code"].str.startswith("DE")]["value_disagg"]

true_values_es = merged_df_mae[merged_df_mae["region_code"].str.startswith("ES")]["value_true"]
disagg_values_es = merged_df_mae[merged_df_mae["region_code"].str.startswith("ES")]["value_disagg"]

mae_de = mean_absolute_error(true_values_de, disagg_values_de).round(2)
mae_es = mean_absolute_error(true_values_es, disagg_values_es).round(2)


In [ ]:
de_total = "223.99e6"
es_total = "40.83e6"

In [ ]:
fig_path = os.path.join("..", "..", "figures", "validation", "validation_commerce_fec.png")


plot_validation_data(validation_data, disagg_data, 
                     lau_shp, de_total, es_total, 
                     mae_de, mae_es, "MWh", "Hotmaps", fig_path)